#Analyze results produced by run_wofost_phenology.py        

In [25]:
import geopandas as gpd
from pathlib import Path
import pandas as pd

read data

In [18]:
cwd = Path.cwd().absolute()
output_dir = cwd.parent.joinpath('results')
print(output_dir)

c:\Users\F80840370\projects\scClim\wofost\winter_wheat_phenology\results


In [29]:
data=[]
for year in range(2010,2022,1):
    data.append(gpd.read_file(output_dir.joinpath(f'results_ww_gs_{year-1}-{year}.gpkg')))

df_all=pd.concat(data,axis=0)

In [39]:
df_all['maturity_doy']=df_all.maturity_date.d
df_all.groupby(['id']).maturity_date.mean()

TypeError: agg function failed [how->mean,dtype->object]

In [43]:
df_all.maturity_date.values

AttributeError: 'numpy.ndarray' object has no attribute 'dayofyear'